In [63]:
import uuid

from apispec import APISpec
from apispec_webframeworks.flask import FlaskPlugin
from flask import Flask


# Create an APISpec
spec = APISpec(
    title="Swagger Petstore",
    version="1.0.0",
    openapi_version="3.0.2",
    plugins=[FlaskPlugin()],
)


# Optional Flask support
app = Flask(__name__)


@app.route("/random")
def random_animal():
    """ Cool Foo-Bar route.
    ---
    get:
        summary: Foo-Bar endpoint.
        description: Get a single foo with the bar ID.
        parameters:
            - name: bar_id
              in: path
              description: Bar ID
              schema:
                  type: python Dict[str,Any]
                  required: true
            - name: aggFunc
              in: query
              description: The funcs arfe gging
              schema:
                  type: string
              default: empty
            - name: test
              schema:
                  type: custom
              in: query
              description: this is just a test
        responses:
            200:
                description: Foo object to be returned.
                schema: FooSchema
            404:
                description: Foo not found.
    """
    # Hardcoded example data
    pet_data = {
        "name": "sample_pet_" + str(uuid.uuid1()),
        "categories": [{"id": 1, "name": "sample_category"}],
    }
    return pet_data


@app.route("/person")
def random_person():
    """ Cool Foo-Bar route.
    ---
    get:
        summary: Foo-Bar endpoint.
        description: Get a single foo with the bar ID.
        parameters:
            - in: query
              name: offset
              schema:
                type: integer
              description: The number of items to skip before starting to collect the result set
            - in: query
              name: limit
              schema:
                type: integer
              description: The numbers of items to return
    """
    return {"message":"this"}
# Register the path and the entities within it
with app.test_request_context():
    spec.path(view=random_animal)
    spec.path(view=random_person)

In [64]:
swagger_ui = """<!DOCTYPE html>
<html>
  <head>
    <title>Swagger</title>
    <meta charset="utf-8"/>
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <link rel="stylesheet" type="text/css" href="//unpkg.com/swagger-ui-dist@3/swagger-ui.css" />
  </head>
  <body>
    <div id="swagger-ui"></div>
    <script src="//unpkg.com/swagger-ui-dist@3/swagger-ui-bundle.js"></script>
    <script>
    const ui = SwaggerUIBundle({
        url: "/swagger",
        dom_id: '#swagger-ui',
        presets: [
          SwaggerUIBundle.presets.apis,
          SwaggerUIBundle.SwaggerUIStandalonePreset
        ],
        layout: "BaseLayout",
        requestInterceptor: (request) => {
          request.headers['X-CSRFToken'] = "{{ csrf_token }}"
          return request;
        }
      })
    </script>
  </body>
</html>"""

@app.route("/")
def read_swagger():
    return swagger_ui


import json
@app.route("/swagger")
def serve_swagger():
    return json.dumps(spec.to_dict())

In [65]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Mar/2020 21:01:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2020 21:01:08] "GET /swagger HTTP/1.1" 200 -
